In [47]:
import os
fdel = os.path.sep
import sys
wd = %pwd
parent = os.path.dirname(wd)

sys.path.append(parent)
import altair as alt
import pandas as pd
print(wd)
from backend import btc_etfs

# data = etf.btc_etf_data(metric = "btc_etf_aum", export_response = True)
# data.df
#data_2 = etf.btc_etf_data(metric = "btc_etf_aum_hist", export_response = True)
# data = etf.json_file_io(filename= wd + fdel + "backend"+fdel+"btc_etf_aum_hist_theblock.json")
# data["chart"]["jsonFile"]["Series"]

/home/purpy-furcat/Documents/Code/New_Endeavours/BTC_ETF_Tracker/backend


In [ ]:
from bs4 import BeautifulSoup
import requests

def get_html_save(url: str, save_path: str = wd+fdel+'last_request.html', save: bool = True):
    r = requests.get(url)
    if save:
        with open(save_path, 'w') as wp:
            wp.write(r.text)
    return r.text  

#html = get_html_save("https://farside.co.uk/?p=997", save=True)
with open(wd+fdel+'last_request.html', 'r') as wp:
    html = wp.read()
soup = BeautifulSoup(html, features="html.parser"); dumpstr = ""

tag = soup.table 
for span in tag.find_all('span'):
    dumpstr+=span.get_text().strip()+"\n"
liststr = dumpstr.split("\n")  
dropafter = liststr.index("Average")
liststr = liststr[:dropafter]
index = []; vals = []; colnames = []; notmatched = []; totalcount = 0
print(liststr)

In [ ]:

# data_melted = data.reset_index(drop=False).melt('Timestamp', var_name='Fund', value_name='Net Flow (USD)')
# data_melted
# Create a bar chart for each column and layer them on top of each other
data_melted = aum_data.reset_index().melt('Timestamp', var_name='Category', value_name='Values') 
data_melted
chart = alt.Chart(data_melted).mark_bar().encode(
    x=alt.X('Timestamp:T', axis=alt.Axis(title='Date')),
    y=alt.Y('Values:Q'),
    color='Category:N',
    # tooltip=['Category']
).interactive()
chart
# chart = alt.Chart(data).mark_bar().encode(x = 'Timestamp', y = 'GBTC').interactive()
# chart

alt.Chart(...)

In [18]:
#Get data on daily flows ######
hybrid_df, last_block_day = btc_etfs.get_hybrid_flows_table()
hybrid_df.index.rename('Date', inplace=True)
hybrid_df /= 1000000  # Convert to millions of USD
net_flow = hybrid_df.sum(axis=1).rename('Net flow total (USD)')
net_flow = net_flow.reset_index()

Requesting data from theblock for etf_flows..
200
Data successfully retrieved.
Dataset:  Spot Bitcoin ETF Flows 
Start:  2024-01-11 00:00:00 
Updated last:  2024-07-29 06:18:04


/home/purpy-furcat/Documents/Code/New_Endeavours/BTC_ETF_Tracker/backend/btc_etfs.py:262: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  flows_part = flows_part.replace('-', np.nan).replace(",", "", regex=True)


In [43]:
window = 20
net_flow[str(window) + "_MA"] = net_flow['Net flow total (USD)'].rolling(window=window).mean()

# Create the bar chart
bar_chart = alt.Chart(net_flow).mark_bar(opacity=0.8).encode(
    x=alt.X('Date:T', axis=alt.Axis(title='Date')),
    y=alt.Y('Net flow total (USD):Q', axis=alt.Axis(title='USD (millions)')),
    color=alt.value('skyblue')
).properties(
    title='Net Flow and Moving Average'
)

# Create the bar chart
line_chart = alt.Chart(net_flow).mark_line().encode(
    x=alt.X('Date:T', axis=alt.Axis(title='Date')),
    y=alt.Y(str(window) + "_MA:Q", axis=alt.Axis(title='USD (millions)')),
    color=alt.value('red')
).properties(
    title='Net Flow and Moving Average'
)


# Combine the charts
combined_chart = alt.layer(bar_chart, line_chart)

combined_chart.show()

alt.LayerChart(...)

In [50]:
from vega_datasets import data

stocks = data.stocks()
google = stocks.query("symbol=='GOOG'")
others = stocks.query("symbol!='GOOG'")

chart = alt.LayerChart(stocks).transform_calculate(
    company = 'datum.symbol == "GOOG" ? "goog" : "competitors"'
)

google = alt.Chart().transform_filter(
    alt.datum.company == "goog"
).mark_bar().encode(
    x='year(date):N',
    y='price:Q',
).properties(title='Google stock prices')

others = alt.Chart().transform_filter(
    alt.datum.company == "competitors"
).mark_line(color='red').encode(
    x='year(date):N',
    y='average(price):Q',
    color=alt.Color('company:N',
                    scale=alt.Scale(range=['red']))
)
chart + google + others
# chart.show()

alt.LayerChart(...)

In [39]:
import plotly.express as px

# Melt the dataframe
melted_df = net_flow.melt(id_vars='Date', value_vars=['Net flow total (USD)', '20_MA'])

# Create the chart
chart = px.line(melted_df, x='Date', y='value', color='variable', title='Net Flow and Moving Average')

# Update layout
chart.update_layout(
    yaxis_title='USD (millions)',
    legend_title='Column'
)

# Add range slider
chart.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1, label="1m", step="month", stepmode="backward"),
                dict(count=6, label="6m", step="month", stepmode="backward"),
                dict(count=1, label="YTD", step="year", stepmode="todate"),
                dict(count=1, label="1y", step="year", stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(visible=True),
        type="date"
    )
)

# # Add bar chart for 'Net flow total (USD)'
# chart.add_bar(melted_df[melted_df['variable'] == 'Net flow total (USD)'], x='Date', y='value')

chart.show()